# Exercício 1

In [58]:
def acao(destino, custo):
    return { 'destino': destino, 'custo': custo}

estados_romenia = [
    { 'estado': 'Arad',
      'acoes': [acao('Zerind', 75), acao('Sibiu', 140), acao('Timisoara', 118)]},

    { 'estado': 'Zerind',
      'acoes': [acao('Arad', 75), acao('Oradea', 71)]},

    { 'estado': 'Timisoara',
      'acoes': [acao('Arad', 118), acao('Lugoj', 111)]},

    { 'estado': 'Sibiu',
      'acoes': [acao('Arad', 140), acao('Oradea', 151), acao('Fagaras', 99),
                acao('Rimnicu Vilcea', 80)]},

    { 'estado': 'Oradea',
      'acoes': [acao('Zerind', 71), acao('Sibiu', 151)]},

    { 'estado': 'Lugoj',
      'acoes': [acao('Timisoara', 111), acao('Mehadia', 70)]},

    { 'estado': 'Mehadia',
      'acoes': [acao('Lugoj', 70), acao('Drobeta', 75)]},

    { 'estado': 'Drobeta',
      'acoes': [acao('Mehadia', 75), acao('Craiova', 120)]},

    { 'estado': 'Craiova',
      'acoes': [acao('Drobeta', 120), acao('Rimnicu Vilcea', 146),
                acao('Pitesti', 138)]},

    { 'estado': 'Rimnicu Vilcea',
      'acoes': [acao('Sibiu', 80), acao('Craiova', 146), acao('Pitesti', 97)]},

    { 'estado': 'Fagaras',
      'acoes': [acao('Sibiu', 99), acao('Bucharest', 211)]},

    { 'estado': 'Pitesti',
      'acoes': [acao('Rimnicu Vilcea', 97), acao('Craiova', 138), acao('Bucharest', 101)]},

    { 'estado': 'Giurgiu',
      'acoes': [acao('Bucharest', 90)]},

    { 'estado': 'Bucharest',
      'acoes': [acao('Fagaras', 211), acao('Pitesti', 101), acao('Giurgiu', 90),
                acao('Urziceni', 85)]},

    { 'estado': 'Urziceni',
      'acoes': [acao('Bucharest', 85), acao('Vaslui', 142), acao('Hirsova', 98)]},

    { 'estado': 'Hirsova',
      'acoes': [acao('Urziceni', 98), acao('Eforie', 86)]},

    { 'estado': 'Eforie',
      'acoes': [acao('Hirsova', 86)]},

    { 'estado': 'Vaslui',
      'acoes': [acao('Urziceni', 142), acao('Iasi', 92)]},

    { 'estado': 'Iasi',
      'acoes': [acao('Vaslui', 92), acao('Neamt', 87)]},

    { 'estado': 'Neamt',
      'acoes': [acao('Iasi', 87)]}
]

heuristica = {
    'Arad': 366,
    'Bucharest': 0,
    'Craiova': 160,
    'Drobeta': 242,
    'Eforie': 161,
    'Fagaras': 176,
    'Giurgiu': 77,
    'Hirsova': 151,
    'Iasi': 226,
    'Lugoj': 244,
    'Mehadia': 241,
    'Neamt': 234,
    'Oradea': 380,
    'Pitesti': 100,
    'Rimnicu Vilcea': 193,
    'Sibiu': 253,
    'Timisoara': 329,
    'Urziceni': 80,
    'Vaslui': 199,
    'Zerind': 374
}

class No:
    #estado é o nome da cidade que eu estou(estado após a ação),
    #acao é ir para a cidade, os dois serão strings
    def __init__(self, estado, custo, funcao_avaliacao, pai, acao):
        self.estado = estado
        self.custo = custo
        self.funcao_avaliacao = funcao_avaliacao
        self.pai = pai
        self.acao = acao

    def __str__(self):
        return f'({self.estado}, {self.custo})'

    def __repr__(self):
        return self.__str__()

    def filhos(self, problema):
        espaco_acoes = next(e for e in problema.espaco_estados if e['estado'] == self.estado)

        resultado = []
        for acao in espaco_acoes['acoes']:
            filho = No(acao['destino'], self.custo + acao['custo'], self.custo + acao['custo'] + heuristica[acao['destino']],
                       self, acao['destino'])
            resultado.append(filho)

        return resultado

    def constroi_solucao(self):
        no_atual = self
        solucao = [no_atual]
        while no_atual.pai != None:
            no_atual = no_atual.pai
            solucao.insert(0, no_atual)

        return solucao

class Problema:
    def __init__(self, espaco_estados, inicial, objetivo): #espaco_estados é uma lista de dicionários
        self.espaco_estados = espaco_estados
        self.inicial = inicial
        self.objetivo = objetivo

BUSCA_INICIANDO = 0
BUSCA_FALHOU = 1
BUSCA_SUCESSO = 2
BUSCA_EM_CURSO = 3

In [59]:
class MinHeap:
    def __init__(self):
        self.contents = []
        self.capacity = 0
        self.size = 0

    # remove o menor elemento do heap e reestabelece a ordem correta
    def remove_min(self):
        if self.size < 1:
            return None

        # guarda o menor elemento e coloca o ultimo elemento na raiz
        minimo = self.contents[0]
        self.contents[0] = self.contents[self.size-1]
        self.size -= 1

        # reestabelece a propriedade do min-heap
        self.__min_heapify(0)

        return minimo

    def adiciona(self, node):
        indice = self.size
        if self.capacity == self.size:
            self.contents.append(node)
            self.capacity += 1
        self.__insert(indice, node)
        self.size += 1


    # Metodos privados
    def __pai(self, i):
        return (i - 1) // 2

    def __filho_esquerdo(self, i):
        return i * 2 + 1

    def __filho_direito(self, i):
        return i * 2 + 2

    def __swap(self, i, j):
        self.contents[i], self.contents[j] = self.contents[j], self.contents[i]

    def __min_heapify(self, i):
        l = self.__filho_esquerdo(i)
        r = self.__filho_direito(i)

        # encontra qual o menor dos tres nos: i, l ou r
        minimo = i

        if l < self.size and self.contents[i].funcao_avaliacao > self.contents[l].funcao_avaliacao:
            minimo = l

        if r < self.size and self.contents[minimo].funcao_avaliacao > self.contents[r].funcao_avaliacao:
            minimo = r

        # se i nao for o menor no, troca de lugar com o menor e continua
        # recursivamente
        if minimo != i:
            self.__swap(i, minimo)
            self.__min_heapify(minimo)

    def __insert(self, i, node):
        self.contents[i] = node
        while i > 0 and self.contents[self.__pai(i)].funcao_avaliacao > self.contents[i].funcao_avaliacao:
            self.__swap(i, self.__pai(i))
            i = self.__pai(i)


# uma fila de prioridade que retorna os nos de valor minimo primeiro
class PriorityQueue:
    def __init__(self):
        self.heap = MinHeap()

    def __iter__(self):
        return iter(self.heap.contents) #Adicionei para que se torne iteravel

    def remove_min(self):
        return self.heap.remove_min() #Adicionei o return

    def adiciona(self, node):
        self.heap.adiciona(node)

In [60]:
class BuscaAStar:
    def __init__(self, problema):
        self.problema = problema
        self.fronteira = PriorityQueue()
        self.fronteira.adiciona(problema.inicial) #nós que estão na fila para serem analisados a cada iteração
        self.visitados = [problema.inicial.estado]
        self.solucao = []
        self.situacao = BUSCA_INICIANDO

    def executar(self):
        count = 1
        while self.situacao != BUSCA_FALHOU and self.situacao != BUSCA_SUCESSO:
            #print(f"A busca esta no No {self.fronteira[0].estado}", end = " ")
            self.passo_busca()
            if self.situacao == BUSCA_SUCESSO:
                #print("e", end = " ")
                break
            else:
              #print(f"e vai para ({count}):",self.fronteira)
              count += 1

        if self.situacao == BUSCA_FALHOU:
            print("Busca falhou")
        elif self.situacao == BUSCA_SUCESSO:
            print("\nSolução Final")
            for estado in self.solucao:
              if estado == self.solucao[-1]:
                print(f"{estado.estado}")
              else:
                print(f"{estado.estado} ->", end=" ")
            print(f"Custo Total: {self.solucao[-1].custo}")

        return

    def passo_busca(self):
        if (self.situacao == BUSCA_FALHOU):
            print("Busca falhou")
            return

        if (self.situacao == BUSCA_SUCESSO):
            print("Busca chegou ao objetivo com sucesso")
            return

        try:
            no = self.fronteira.remove_min()
        except IndexError:
            self.situacao = BUSCA_FALHOU
            return

        # faz teste do objetivo
        if self.problema.objetivo(no):
            self.situacao = BUSCA_SUCESSO
            self.solucao = no.constroi_solucao()
            return

        # obtem os filhos do no
        for filho in no.filhos(self.problema):
            if not (filho in self.fronteira) and not (filho.estado in self.visitados):
                print(f"Saindo de {no.estado} para {filho.estado}")
                self.fronteira.adiciona(filho)
                self.visitados.append(filho.estado)

        return

# Exercício 2

In [61]:
no_arad = No('Arad', 0,366, None, None)
problema_romenia = Problema(estados_romenia,
                            no_arad,
                            lambda no: no.estado == 'Bucharest')

print("Busca em largura da cidade de Arad ate Bucharest:")
busca = BuscaAStar(problema_romenia)
busca.executar()

Busca em largura da cidade de Arad ate Bucharest:
Saindo de Arad para Zerind
Saindo de Arad para Sibiu
Saindo de Arad para Timisoara
Saindo de Sibiu para Oradea
Saindo de Sibiu para Fagaras
Saindo de Sibiu para Rimnicu Vilcea
Saindo de Rimnicu Vilcea para Craiova
Saindo de Rimnicu Vilcea para Pitesti
Saindo de Fagaras para Bucharest
Saindo de Timisoara para Lugoj

Solução Final
Arad -> Sibiu -> Fagaras -> Bucharest
Custo Total: 450


# Exercício 3

In [62]:
no_lugoj = No('Lugoj', 0, 244, None, None)
problema_romenia = Problema(estados_romenia, no_lugoj, lambda no: no.estado == 'Bucharest')
print("Rota de Lugoj até Bucharest:")
busca = BuscaAStar(problema_romenia)
busca.executar()

Rota de Lugoj até Bucharest:
Saindo de Lugoj para Timisoara
Saindo de Lugoj para Mehadia
Saindo de Mehadia para Drobeta
Saindo de Drobeta para Craiova
Saindo de Craiova para Rimnicu Vilcea
Saindo de Craiova para Pitesti
Saindo de Timisoara para Arad
Saindo de Pitesti para Bucharest

Solução Final
Lugoj -> Mehadia -> Drobeta -> Craiova -> Pitesti -> Bucharest
Custo Total: 504


In [63]:
no_zerind = No("Zerind", 0, 374, None, None)
problema_romenia = Problema(estados_romenia, no_zerind, lambda no: no.estado == 'Bucharest')
print("Rota de Zerind até Bucharest:")
busca = BuscaAStar(problema_romenia)
busca.executar()

Rota de Zerind até Bucharest:
Saindo de Zerind para Arad
Saindo de Zerind para Oradea
Saindo de Arad para Sibiu
Saindo de Arad para Timisoara
Saindo de Sibiu para Fagaras
Saindo de Sibiu para Rimnicu Vilcea
Saindo de Rimnicu Vilcea para Craiova
Saindo de Rimnicu Vilcea para Pitesti
Saindo de Fagaras para Bucharest
Saindo de Timisoara para Lugoj

Solução Final
Zerind -> Arad -> Sibiu -> Fagaras -> Bucharest
Custo Total: 525


# Exercício 4

In [64]:
heuristica = {
    'Arad': 543.60,
    'Bucharest': 149.06,
    'Craiova': 331.43,
    'Drobeta':  418.15,
    'Eforie': 90.59,
    'Fagaras': 265.57,
    'Giurgiu': 179.92,
    'Hirsova': 0,
    'Iasi': 275.85,
    'Lugoj': 486.35,
    'Mehadia': 440.93,
    'Neamt': 280.71,
    'Oradea':  534.89,
    'Pitesti': 243.58,
    'Rimnicu Vilcea': 285.39,
    'Sibiu': 322.40,
    'Timisoara': 540.39,
    'Urziceni': 103.02,
    'Vaslui':  217.64,
    'Zerind':  543.67
}

class No:
    #estado é o nome da cidade que eu estou(estado após a ação),
    #acao é ir para a cidade, os dois serão strings
    def __init__(self, estado, custo, funcao_avaliacao, pai, acao):
        self.estado = estado
        self.custo = custo
        self.funcao_avaliacao = funcao_avaliacao
        self.pai = pai
        self.acao = acao

    def __str__(self):
        return f'({self.estado}, {self.custo})'

    def __repr__(self):
        return self.__str__()

    def filhos(self, problema):
        espaco_acoes = next(e for e in problema.espaco_estados if e['estado'] == self.estado)

        resultado = []
        for acao in espaco_acoes['acoes']:
            filho = No(acao['destino'], self.custo + acao['custo'], self.custo + acao['custo'] + heuristica[acao['destino']],
                       self, acao['destino'])
            resultado.append(filho)

        return resultado

    def constroi_solucao(self):
        no_atual = self
        solucao = [no_atual]
        while no_atual.pai != None:
            no_atual = no_atual.pai
            solucao.insert(0, no_atual)

        return solucao

class Problema:
    def __init__(self, espaco_estados, inicial, objetivo): #espaco_estados é uma lista de dicionários
        self.espaco_estados = espaco_estados
        self.inicial = inicial
        self.objetivo = objetivo

BUSCA_INICIANDO = 0
BUSCA_FALHOU = 1
BUSCA_SUCESSO = 2
BUSCA_EM_CURSO = 3

class MinHeap:
  def __init__(self):
    self.contents = []
    self.capacity = 0
    self.size = 0

  # remove o menor elemento do heap e reestabelece a ordem correta
  def remove_min(self):
    if self.size < 1:
        return None

    # guarda o menor elemento e coloca o ultimo elemento na raiz
    minimo = self.contents[0]
    self.contents[0] = self.contents[self.size-1]
    self.size -= 1

    # reestabelece a propriedade do min-heap
    self.__min_heapify(0)

    return minimo

  def adiciona(self, node):
    indice = self.size
    if self.capacity == self.size:
        self.contents.append(node)
        self.capacity += 1
    self.__insert(indice, node)
    self.size += 1


  # Metodos privados
  def __pai(self, i):
    return (i - 1) // 2

  def __filho_esquerdo(self, i):
    return i * 2 + 1

  def __filho_direito(self, i):
    return i * 2 + 2

  def __swap(self, i, j):
    self.contents[i], self.contents[j] = self.contents[j], self.contents[i]

  def __min_heapify(self, i):
    l = self.__filho_esquerdo(i)
    r = self.__filho_direito(i)

    # encontra qual o menor dos tres nos: i, l ou r
    minimo = i

    if l < self.size and self.contents[i].funcao_avaliacao > self.contents[l].funcao_avaliacao:
        minimo = l

    if r < self.size and self.contents[minimo].funcao_avaliacao > self.contents[r].funcao_avaliacao:
        minimo = r

    # se i nao for o menor no, troca de lugar com o menor e continua
    # recursivamente
    if minimo != i:
        self.__swap(i, minimo)
        self.__min_heapify(minimo)

  def __insert(self, i, node):
    self.contents[i] = node
    while i > 0 and self.contents[self.__pai(i)].funcao_avaliacao > self.contents[i].funcao_avaliacao:
        self.__swap(i, self.__pai(i))
        i = self.__pai(i)

no_zerind = No("Zerind", 0, 543.67, None, None)
problema_romenia = Problema(estados_romenia, no_zerind, lambda no: no.estado == 'Hirsova')
print("Rota de Zerind até Hirsova:")
busca = BuscaAStar(problema_romenia)
busca.executar()

Rota de Zerind até Hirsova:
Saindo de Zerind para Arad
Saindo de Zerind para Oradea
Saindo de Oradea para Sibiu
Saindo de Sibiu para Fagaras
Saindo de Sibiu para Rimnicu Vilcea
Saindo de Fagaras para Bucharest
Saindo de Rimnicu Vilcea para Craiova
Saindo de Rimnicu Vilcea para Pitesti
Saindo de Arad para Timisoara
Saindo de Bucharest para Giurgiu
Saindo de Bucharest para Urziceni
Saindo de Urziceni para Vaslui
Saindo de Urziceni para Hirsova

Solução Final
Zerind -> Oradea -> Sibiu -> Fagaras -> Bucharest -> Urziceni -> Hirsova
Custo Total: 715


Exercicio 5

In [65]:
import math
import heapq

# Problema de distribuição de água adaptado a busca A*
def busca_a_estrela(grafo, inicio, objetivo):
    fila_prioridade = [(0, inicio)]
    custo_g = {inicio: 0}
    caminho = {inicio: None}  # Dicionário para rastrear o caminho percorrido

    while fila_prioridade:
        custo_atual, atual = heapq.heappop(fila_prioridade)
        
        if atual == objetivo:
            # Caminho encontrado
            caminho_reverso = [atual]
            while caminho[atual] is not None:
                atual = caminho[atual]
                caminho_reverso.append(atual)
            caminho_reverso.reverse()
        
            return caminho_reverso, custo_g[objetivo]
        
        print("Vizinhos do ", atual, " disponiveis: ", grafo.get(atual, []))
        for vizinho in grafo.get(atual, []):
            novo_custo = custo_g[atual] + heuristica(atual, vizinho)
            print("custo de ir para ", vizinho, ":", novo_custo)
            if vizinho not in custo_g or novo_custo < custo_g[vizinho]:
                custo_g[vizinho] = novo_custo
                prioridade = novo_custo + heuristica(vizinho, objetivo)
                print("prioridade do vizinho ", vizinho, ":", prioridade)
                heapq.heappush(fila_prioridade, (prioridade, vizinho))
                caminho[vizinho] = atual
        print('\nFila de prioridade atual: ',fila_prioridade, '\n')
    
    return None

grafo = {
    'D1': ['C1', 'C2', 'C3'],
    'D2': ['C2', 'C4'],
    'D3': ['C1', 'C5'],
    'D4': ['C4', 'C5', 'C2'],
    'C1': ['D1', 'D3'],
    'C2': ['D4', 'D1', 'D2'],
    'C3': ['D1'],
    'C4': ['D4', 'D2'],
    'C5': ['D3', 'D4']
}

# Coordenadas dos pontos de distribuição e consumidores
coordenadas = {
    'D1': (0, 0),
    'D2': (2, 0),
    'D3': (0, 2),
    'D4': (2, 2),
    'C1': (1, 1),
    'C2': (-1, 3),
    'C3': (4, 3),
    'C4': (5, 1),
    'C5': (3, 6)
}

# Função de heurística usando a distância euclidiana
def heuristica(ponto1, ponto2):
    x1, y1 = coordenadas[ponto1]
    x2, y2 = coordenadas[ponto2]
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

# Exemplo de uso
caminho = busca_a_estrela(grafo, 'C1', 'C4')
if caminho:
    print("Caminho encontrado:", caminho)
else:
    print("Caminho não encontrado")


Vizinhos do  C1  disponiveis:  ['D1', 'D3']
custo de ir para  D1 : 1.4142135623730951
prioridade do vizinho  D1 : 6.513233075965879
custo de ir para  D3 : 1.4142135623730951
prioridade do vizinho  D3 : 6.513233075965879

Fila de prioridade atual:  [(6.513233075965879, 'D1'), (6.513233075965879, 'D3')] 

Vizinhos do  D1  disponiveis:  ['C1', 'C2', 'C3']
custo de ir para  C1 : 2.8284271247461903
custo de ir para  C2 : 4.576491222541475
prioridade do vizinho  C2 : 10.901046542878234
custo de ir para  C3 : 6.414213562373095
prioridade do vizinho  C3 : 8.650281539872886

Fila de prioridade atual:  [(6.513233075965879, 'D3'), (10.901046542878234, 'C2'), (8.650281539872886, 'C3')] 

Vizinhos do  D3  disponiveis:  ['C1', 'C5']
custo de ir para  C1 : 2.8284271247461903
custo de ir para  C5 : 6.414213562373095
prioridade do vizinho  C5 : 11.799378369507599

Fila de prioridade atual:  [(8.650281539872886, 'C3'), (10.901046542878234, 'C2'), (11.799378369507599, 'C5')] 

Vizinhos do  C3  disponivei

![Alt text](image.png)

![Alt text](image-1.png)